In [19]:
import pandas as pd


data = pd.read_csv('./data/testPreprocessed.csv')
attrs = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
tags = list()
'''
for i in range(data['toxic'].size):
    tag = ''
    for a in attrs:
        if(data.iloc[i][a] == 1):
            tag += a + '-AND-'
    if(tag == ''):
        tag = 'normal'
    tags.append(tag)
data['tag'] = pd.Series(tags)
data.to_csv('./data/testPreprocessed.csv', index=False)
'''

In [ ]:
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import pandas as pd
import tqdm

data = pd.read_csv('./traintestData/trainPreprocessed.csv')

for i in range(data['id'].size):
    tempstr = data['CommentTxtWakati'][i].replace('[', '')
    tempstr = tempstr.replace(']', '')
    tempstr = tempstr.replace('\'', '')
    tempstr = tempstr.replace(',', '')
    tempstr = tempstr.split()
    vocab.append(tempstr)

# traindataの準備
trainData = [TaggedDocument(words=vocab[i], tags=data.iloc[i]['tag']) for i in tqdm.tqdm(range(data['id'].size))]

# learn
print('start learning')
model = Doc2Vec(documents=trainData, size=400, min_count=5, iter=30)
model.save('./model/d2v.model')
print('finished')


100%|████████████████████████████████████████████████████████████████████████| 159571/159571 [00:24<00:00, 6628.11it/s]

start learning


In [20]:
import pandas as pd

train = pd.read_csv('./traintestData/trainPreprocessed.csv')

vocab = list()
for i in range(data['id'].size):
    tempstr = data['CommentTxtWakati'][i].replace('[', '')
    tempstr = tempstr.replace(']', '')
    tempstr = tempstr.replace('\'', '')
    tempstr = tempstr.replace(',', '')
    tempstr = tempstr.split()
    vocab.append(tempstr)

sample = pd.Series(vocab)
sample.head()

0    [explanation, edit, make, username, Hardcore, ...
1    [match, background, colour, stick, thanks, tal...
2    [try, edit, war, guy, remove, relevant, inform...
3    [make, real, suggestion, improvement, wonder, ...
4                  [sir, hero, chance, remember, page]
dtype: object

In [1]:
from gensim.models.word2vec import Word2Vec
import pandas as pd

train = pd.read_csv('./traintestData/trainPreprocessed.csv')
test = pd.read_csv('./traintestData/testPreprocessed.csv')

vocab = list()
for j in range(2):
    if(j == 0):
        data = train
    elif(j == 1):
        data = test
    for i in range(data['id'].size):
        tempstr = data['CommentTxtWakati'][i].replace('[', '')
        tempstr = tempstr.replace(']', '')
        tempstr = tempstr.replace('\'', '')
        tempstr = tempstr.replace(',', '')
        tempstr = tempstr.split()
        vocab.append(tempstr)

# learn
print('start learning')
model = Word2Vec(sentences=vocab, size=50, window=5, min_count=5, workers=4, iter=50)
model.save('./model/w2v.model')
print('finished')


C:\Users\033041004\AppData\Local\Continuum\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


start learning
finished


In [ ]:
from gensim.models.word2vec import Word2Vec
import pandas as pd
from sklearn.model_selection import train_test_split
import pickle
import numpy as np


# 単語をW2Vで特徴量ベクトルに変換してPickle化
model = Word2Vec.load('./model/w2v.model')
train = pd.read_csv('./traintestData/trainPreprocessed.csv')
test = pd.read_csv('./traintestData/testPreprocessed.csv')

data = train
vocabTr = list()
for i in range(data['id'].size):
    tempstr = data['CommentTxtWakati'][i].replace('[', '')
    tempstr = tempstr.replace(']', '')
    tempstr = tempstr.replace('\'', '')
    tempstr = tempstr.replace(',', '')
    tempstr = tempstr.split()
    vocabTr.append(tempstr)
data = test
vocabTe = list()
for i in range(data['id'].size):
    tempstr = data['CommentTxtWakati'][i].replace('[', '')
    tempstr = tempstr.replace(']', '')
    tempstr = tempstr.replace('\'', '')
    tempstr = tempstr.replace(',', '')
    tempstr = tempstr.split()
    vocabTe.append(tempstr)

# train data
trainList = list()
for l in vocabTr:
    fv = np.array([model.wv[d]  for d in l if d in model.wv.vocab])
    trainList.append(fv)
with open('./traintestData/trainVocab.pickle', mode='wb') as f:
    pickle.dump(f, trainList)

# test data
testList = list()
for l in vocabTe:
    fv = np.array([model.wv[d]  for d in l if d in model.wv.vocab])
    testList.append(fv)
with open('./traintestData/testVocab.pickle', mode='wb') as f:
    pickle.dump(f, testList)

print('finished')

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split


train = pd.read_csv('./traintestData/trainPreprocessed.csv')
test = pd.read_csv('./traintestData/testPreprocessed.csv')
attrs = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

import collections

ct = collections.Counter(train['tag'])

In [29]:
import pandas as pd
import random
import pickle


trainData = pd.read_csv('./traintestData/trainPreprocessed.csv')

# 対象のIDを取得:validId
extracted = trainData[trainData['tag'] == 'normal']
extracted = extracted.reset_index(drop=True)
sidx = random.sample(range(extracted.shape[0]), 16225)
sidx = sorted(sidx)

validId = list()
for i in range(extracted.shape[0]):
    if(i in sidx):
        validId.append(extracted['id'][i])
abnormal = trainData[trainData['tag'] != 'normal']
abnormal = abnormal.reset_index(drop=True)
for i in range(abnormal.shape[0]):
    validId.append(abnormal['id'][i])

# validIdに入っているデータのみ使用する
with open('./traintestData/trainVocabW2VSUM.pickle', mode='rb') as f:
    trainList = pickle.load(f)
with open('./traintestData/trainLabel.pickle', mode='rb') as f:
    label = pickle.load(f)

validTrainList = list()
validLabel = list()
for i in range(trainData.shape[0]):
    if(trainData['id'][i] in validId):
        validTrainList.append(trainList[i])
        validLabel.append(label[i])
validTrainList = np.array(validTrainList)
validLabel = np.array(validLabel)


(32450, 50)
(32450, 6)


# memo

## タグの割合
- trainデータ
    - normal:143346
    - other:16225
Counter({'identity_hate-AND-': 54,
         'insult-AND-': 301,
         'insult-AND-identity_hate-AND-': 28,
         'normal': 143346,
         'obscene-AND-': 317,
         'obscene-AND-identity_hate-AND-': 3,
         'obscene-AND-insult-AND-': 181,
         'obscene-AND-insult-AND-identity_hate-AND-': 18,
         'obscene-AND-threat-AND-': 2,
         'obscene-AND-threat-AND-insult-AND-': 2,
         'threat-AND-': 22,
         'threat-AND-insult-AND-': 3,
         'toxic-AND-': 5666,
         'toxic-AND-identity_hate-AND-': 136,
         'toxic-AND-insult-AND-': 1215,
         'toxic-AND-insult-AND-identity_hate-AND-': 134,
         'toxic-AND-obscene-AND-': 1758,
         'toxic-AND-obscene-AND-identity_hate-AND-': 35,
         'toxic-AND-obscene-AND-insult-AND-': 3800,
         'toxic-AND-obscene-AND-insult-AND-identity_hate-AND-': 618,
         'toxic-AND-obscene-AND-threat-AND-': 11,
         'toxic-AND-obscene-AND-threat-AND-insult-AND-': 131,
         'toxic-AND-obscene-AND-threat-AND-insult-AND-identity_hate-AND-': 56,
         'toxic-AND-severe_toxic-AND-': 41,
         'toxic-AND-severe_toxic-AND-identity_hate-AND-': 3,
         'toxic-AND-severe_toxic-AND-insult-AND-': 14,
         'toxic-AND-severe_toxic-AND-insult-AND-identity_hate-AND-': 7,
         'toxic-AND-severe_toxic-AND-obscene-AND-': 158,
         'toxic-AND-severe_toxic-AND-obscene-AND-identity_hate-AND-': 6,
         'toxic-AND-severe_toxic-AND-obscene-AND-insult-AND-': 989,
         'toxic-AND-severe_toxic-AND-obscene-AND-insult-AND-identity_hate-AND-': 265,
         'toxic-AND-severe_toxic-AND-obscene-AND-threat-AND-': 4,
         'toxic-AND-severe_toxic-AND-obscene-AND-threat-AND-insult-AND-': 64,
         'toxic-AND-severe_toxic-AND-obscene-AND-threat-AND-insult-AND-identity_hate-AND-': 31,
         'toxic-AND-severe_toxic-AND-threat-AND-': 11,
         'toxic-AND-severe_toxic-AND-threat-AND-identity_hate-AND-': 1,
         'toxic-AND-severe_toxic-AND-threat-AND-insult-AND-': 1,
         'toxic-AND-threat-AND-': 113,
         'toxic-AND-threat-AND-identity_hate-AND-': 7,
         'toxic-AND-threat-AND-insult-AND-': 16,
         'toxic-AND-threat-AND-insult-AND-identity_hate-AND-': 3})